In [ ]:
# !cp drive/MyDrive/python檔/aicup/run/dataset.py .
# !cp drive/MyDrive/python檔/aicup/run 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.insert(0,"/content/drive/My Drive/python檔/aicup/run")

In [ ]:
# pip install transformers==3
# pip install pytorch-crf

SyntaxError: ignored

In [ ]:
from dataset import bert_stc_dataset
from model2 import model_crf
from train import train
from txt_preprocess2 import preprocess2

from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [ ]:
file_path = '/content/drive/My Drive/python檔/aicup/run/data/train_input.data'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('{} is being used'.format(device))

cuda is being used


In [ ]:
# ---------------前處理---------------
with open(file_path, 'r', encoding='utf-8') as f:
	data=f.readlines()#.encode('utf-8').decode('utf-8-sig')

preprocessor = preprocess2(data)

stcs, original_labels= preprocessor.get_stc_label()
stcs, labels = preprocessor.get_stcs_label2ids()
tag2id_dict = preprocessor.tag2id(original_labels)
n_tags = len(tag2id_dict)
print(tag2id_dict)
print('tags數: {}'.format(n_tags))

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
max_length = max([len(txt) for txt in stcs])

# ---------------模型---------------
model = model_crf(n_tags= n_tags).to(device)

train_x, test_x, train_y, test_y = train_test_split(stcs, labels, test_size= 0.5, shuffle= True, random_state= 42)

train_dataset = bert_stc_dataset(stcs= train_x, labels= train_y, tokenizer= tokenizer, max_length= 400)
print('training stcs 總數: {}'.format(len(train_dataset)))
train_dataloader = DataLoader(train_dataset, batch_size= 128, shuffle= True)

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2)

# ---------------訓練---------------
# train_model = train(model= model, optimizer= optimizer, train_loader= train_dataloader, test_loader= 0, num_epochs= 5, device= device)

sentences總數: 15843
labels總數: 15843
醫師：啊回去還好嗎
['[CLS]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[SEP]']
sentences總數: 15843
labels總數: 15843
醫師：啊回去還好嗎
['[CLS]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[SEP]']
{'B-ID': 0, 'B-clinical_event': 1, 'B-contact': 2, 'B-education': 3, 'B-family': 4, 'B-location': 5, 'B-med_exam': 6, 'B-money': 7, 'B-name': 8, 'B-organization': 9, 'B-profession': 10, 'B-time': 11, 'I-ID': 12, 'I-clinical_event': 13, 'I-contact': 14, 'I-education': 15, 'I-family': 16, 'I-location': 17, 'I-med_exam': 18, 'I-money': 19, 'I-name': 20, 'I-organization': 21, 'I-profession': 22, 'I-time': 23, 'O': 24, '[CLS]': 25, '[SEP]': 26}
tags數: 27



training stcs 總數: 7921


In [ ]:
train_loss = {}
test_loss = {}
train_f1 = {}
test_f1 = {}

for epoch in range(10):

  preds = []
  gts = []
  epoch_loss = 0
  iteration = 0

  model.train()

  for idx, batch_dict in enumerate(train_dataloader):

    print('idx: ',idx+1)
    input_ids = batch_dict['input_ids'].to(device)
    attention_mask = batch_dict['attention_mask'].to(device)
    labels = batch_dict['labels'].to(device)

    model.zero_grad()

    pred_labels = model(input_ids, attention_mask.bool())

    loss = model.neg_log_likelihood(input_ids, attention_mask, labels, reduction= 'mean')
    loss.backward()
    optimizer.step()

    # mask gt labels 
    labels = batch_dict['labels'].numpy()
    masks = batch_dict['attention_mask'].numpy()
    # masked_list = (labels*mask).numpy()

    labels_nopad = []
    for label , seq_mask in zip(labels, masks):

      seq = [tag for tag, mask in zip(label, seq_mask) if mask == 1]
      labels_nopad.append(seq)

    # one dim array 
    preds= [tag for seq in pred_labels for tag in seq]
    gts= [tag for seq in labels_nopad for tag in seq]

    epoch_loss += loss.item()
    iteration += 1

    print('pred_labels', pred_labels)
    print('preds:', preds)
    print('gts:',gts)
    print(f1_score(y_true= gts, y_pred= preds, average= 'macro'))
    print(f1_score(y_true= gts, y_pred= preds, average= 'micro'))
    print(loss.item())

    # print('idx:{} loss:{}'.format(idx, loss))

  f1 = f1_score(y_true= gts, y_pred= preds, average= 'macro')
  avg_loss = epoch_loss / iteration
  scheduler.step()

  print('epoch {}/{} | train_f1 {:.2f} | train_epoch_avg_loss {:.2f}'.format(epoch+1, 10, f1, avg_loss))

idx:  1
pred_labels [[8, 21, 12, 24, 8, 1, 26, 10], [8, 19, 19, 1, 20, 9, 25, 20, 12, 19, 21, 12, 9, 14, 7, 9, 14, 7, 9, 14, 7, 9, 14, 7, 14, 7, 9, 14, 7, 10, 10], [8, 19, 21, 18, 1, 26, 26, 26, 26, 26, 8, 21, 11, 20, 12, 25, 20, 12, 25, 20, 12, 19, 1, 12, 19, 1, 12, 16, 16, 16, 16, 16, 16, 16, 11], [8, 1, 20, 12, 19, 19, 19, 19, 21, 12, 25, 20, 12, 14, 7, 10, 14, 7, 10, 14, 7, 12, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 1, 20, 12, 19, 19, 19, 19, 1, 26, 10], [8, 1, 12, 24, 8, 1, 26, 10], [8, 1, 12, 24, 5, 20, 26, 14, 7, 10], [8, 19, 21, 11, 20, 12, 24, 5, 20, 12, 19, 1, 26, 8], [8, 1, 12, 9, 9, 9, 9, 14, 7, 14, 7, 10, 17, 8, 1, 8, 1, 8, 1, 8, 1, 26, 10], [8, 1, 20, 12, 19, 1, 20, 12, 19, 10, 14, 10, 10, 10, 14, 11], [20, 12, 25, 20, 9, 14, 7, 9, 14, 7, 9, 25, 3, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 26, 26, 26, 26, 10], [20, 12, 24, 8, 1, 20

KeyboardInterrupt: ignored

In [ ]:
# f1_score(y_true= gts, y_pred= preds, average='micro')